In [1]:
# Imports
import pandas as pd
import numpy as np

## display Markdown
from IPython.display import Markdown, display

# os
import os

In [2]:
# set paths

# get current working directory
budget_viz_path = os.getcwd()

# output data path
output_data_path = os.path.join(budget_viz_path, 'output_data/')

## create folder if not exists
if not os.path.exists(output_data_path):
    os.makedirs(output_data_path)

# input data path
input_data_path = os.path.join(budget_viz_path, 'data/ZaИеcznik nr 2/')

single_output_file_name = 'polish_budget.csv'


# # files
files = [
    'ZaИеcznik nr 2 cz od 01 do 82.xls',
    # ! not working
    # 'ZaИеcznik nr 2 cz 83.xls',
    'ZaИеcznik nr 2 cz od 84 do 90.xls',
]

In [3]:
# display LOAD DATA as H1
display(Markdown('<h1>LOAD DATA</h1>'))

# --------------------------------- settings --------------------------------- #

# # show all rows
pd.set_option('display.max_rows', None)

# ------------------------------------- < ------------------------------------ #


# ------------------------------- manipulations ------------------------------ #

def process_file(folder_path, file_name):

    def read_data(folder_path, file_name):

        print(
            f'Reading data from: {folder_path+file_name}'
        )

        df = pd.read_excel(folder_path+file_name, header=None)

        # split file name at 'cz'
        output_file_name = file_name.split('cz')[-1]

        # output file name -> dynamic, take last two numbers from file name
        import re

        # check if there are two numbers after 'cz'
        if len(re.findall(r'\d+', output_file_name)) == 2:
            output_file_name = re.findall(r'\d+', output_file_name)[-2:]

            output_file_name = '_'.join(output_file_name)
            # +.csv
            output_file_name = output_file_name + '.csv'

        else:
            output_file_name = re.findall(r'\d+', output_file_name)[-1]
            # +.csv
            output_file_name = output_file_name + '.csv'

        return df, output_file_name
    
    # read data
    df, output_file_name = read_data(folder_path, file_name)

    def basic_data_manipulations(df):

        # replace column headers with merged value of 4 first rows
        df.columns = df.iloc[0:4, :].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0)

        # drop first 6 rows
        df = df.iloc[7:, :]

        # remove NaNs
        df = df.dropna(axis=0, how='all')

        # select only Część, Dział, Rozdział, Treść and Plan na 2023 r. columns
        df = df.iloc[:, [0, 1, 2, 3, 5]]

        # rename columns to Część, Dział, Rozdział, Treść, Plan
        # ! diff 1 -> implement it
        df.columns = ['Część', 'Dział', 'Rozdział', 'Treść', 'Plan']

        return df

    df = basic_data_manipulations(df)


    # ! diff 2 -> errors at 15/01 Część
    # ensure that Część, Dział, Rozdział values are either NaN or int
    # i still have 83.0 in Część column, convert it to 83 and then to str

    # df['Część'] = df['Część'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)
    # df['Dział'] = df['Dział'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)
    # df['Rozdział'] = df['Rozdział'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)


    def create_sub_dfs(df, columns):
        # create sub dfs based on columns
        # ex. columns = ['Część', 'Dział', 'Rozdział']

        # divide df into multiple dfs based on NaN values in columns Część, Dział, Rozdział, keep only this column and Treść
        dfs = []

        for column in columns:
            dfs.append(df[df[column].notna()][[column, 'Treść']].drop_duplicates().sort_values(by=[column]))

        return dfs

    # create sub dfs based on columns
    sub_dfs = create_sub_dfs(df, ['Część', 'Dział', 'Rozdział'])

    df_czesc = sub_dfs[0]
    df_dzial = sub_dfs[1]
    df_rozdzial = sub_dfs[2]

    # ---------------------------- fill NaNs properly ---------------------------- #

    # ! todo: fill NaNs -> first fill Rozdział whereever there are NaNs in Dział, Część
    # ! todo: then fill Dział whereever there are NaNs in Część
    # ! todo: fill Część whereever


    def fill_columns_general(df, column_to_fill, condition, fill_rule='ffill'):

        if condition is not None:
            if fill_rule == 'ffill':
                df[column_to_fill] = np.where(condition, df[column_to_fill].fillna(method=fill_rule), df[column_to_fill])
            elif fill_rule == 'bfill':
                df[column_to_fill] = np.where(condition, df[column_to_fill].fillna(method=fill_rule), df[column_to_fill])

        else:
            df[column_to_fill] = df[column_to_fill].fillna(method=fill_rule)

        return df

    # fill down Rozdział column, where Dział and Część is empty
    df = fill_columns_general(df, 'Rozdział', df['Dział'].isnull() & df['Część'].isnull(), 'ffill')

    # fill down Dział column, where Część is empty
    df = fill_columns_general(df, 'Dział', df['Część'].isnull(), 'ffill')

    # fill down Część column
    df = fill_columns_general(df, 'Część', None, 'ffill')

    # --------------------------------- leaf path -------------------------------- #


    # change id to string ( Część-Dział-Rozdział ) for values that are not NaN -> check each Część, Dział, Rozdział if it is not NaN, for each non NaN value add it to string and concatenate with '-'

    # create path to each datum -> consisting of Część, Dział, Rozdział column values if they are not NaN
    def create_path_to_datum(path_cells, columns, separator):
        path = ''
        for column in columns:
            if path_cells[column] is not np.nan:

                # if path is not empty, then add separator
                if path != '':
                    path += separator
                path += str(path_cells[column])

        return path

    # create path for each row
    df['id'] = df.apply(lambda x: create_path_to_datum(x, ['Część', 'Dział', 'Rozdział'], '-'), axis=1)

    # ------------------------------------- < ------------------------------------ #

    # -------------------------------- leaf parent ------------------------------- #

    # parent path is path from id, but without last element, unless it is total  whenever there is only one element in path
    # df['parent'] = ['' if x == '' else '-'.join(x.split('-')[:-1]) for x in df['id']]
    df['parent'] = df['id'].apply(lambda x: '-'.join(x.split('-')[:-1]) if len(x.split('-')) > 1 else 'total')

    # ------------------------------------- < ------------------------------------ #

    # -------------------------------- leaf label -------------------------------- #

    # create column with Plan value as label -> if Plan > 1000, then Plan/1000 + ' MLD', else Plan + ' MLN'
    # function: change float value to MLN / MLD depending on how big the input number is
    def dynamic_format(x):
        million = 1000000
        if x > million:
            return str(round(x/1000000, 1)) + ' MLD'
        else:
            # round to two 0 decimals
            return str(round(x/1000, 1)) + ' MLN'
        
    df['label'] = df['Plan'].apply(lambda x: dynamic_format(x))

    # # super label -> Label + Value (For outermost squares)

    # combine Treść and label columns into super_label column, unless both are 'total' then use 'total' only
    df['super_label'] = df.apply(lambda x: x['Treść'] + ' ' + x['label'] if x['Treść'] != 'total' and x['label'] != 'total' else x['Treść'] if x['Treść'] == 'total' else x['label'], axis=1)


    # add row with sum of all values in Plan column for where parent = 'total'; id = 'total', parent = '', Plan = sum of all values in Plan column for where parent = 'total'
    # df_total = pd.DataFrame({'id': ['total'], 'parent': [''], 'label': ['total'], 'Plan': [df.loc[df['parent'] == 'total', 'Plan'].sum()], 'Treść': ['total']})

    # df = df.append(df_total, ignore_index=True)

    # --------------------------- mapping treść columns -------------------------- #

    # add Treść_część based on Część column value and Treść column value in df_czesc
    df['Treść_część'] = df['Część'].map(df_czesc.set_index('Część')['Treść'])
    df['Treść_część'] = df['Treść_część'].fillna(df['Treść'])


    # add Treść_dział based on Dział column value and Treść column value in df_dzial
    df['Treść_dział'] = df['Dział'].map(df_dzial.set_index('Dział')['Treść'])
    df['Treść_dział'] = df['Treść_dział'].fillna(df['Treść'])

    return df, output_file_name


def create_file_if_not_exists(file_path, file_name, df):

    # if file exists, then remove it
    if os.path.isfile(f'{file_path}{file_name}'):
        os.remove(f'{file_path}{file_name}')

        # log removed file
        print(f'Removed file: {file_path}{file_name}')

    # check if file exists
    if not os.path.isfile(f'{file_path}{file_name}'):
        # create new file
        df.to_csv(
            f'{file_path}{file_name}',
            index=False,
            sep=';'
        )

        # print
        print(f'Created file: {file_path}{file_name}')

    return df


# create total row
def create_total_row(df, rule_for_total_row):
    # ex rule = df.loc[df['parent'] == 'total', 'Plan'].sum()

    df_total = pd.DataFrame({'id': ['total'], 'parent': [''], 'label': ['total'], 'Plan': [rule_for_total_row], 'Treść': ['total']})

    df = df.append(df_total, ignore_index=True)

    return df


single_output_file = create_file_if_not_exists(output_data_path, single_output_file_name, pd.DataFrame())

# # run files through function
for file in files:
    
    # process file

    df, output_file_name = process_file(input_data_path, file)

    # create file for each file
    file = create_file_if_not_exists(output_data_path, output_file_name, df)

    # ---------------------- Outputing to polish_budget.csv ---------------------- #

    # append_df_to_single_file(df, single_output_file_name)
    single_output_file = single_output_file.append(df, ignore_index=True)

    # save single output file
    single_output_file.to_csv(
        f'{output_data_path}{single_output_file_name}',
        index=False,
        sep=';'
    )

    # ------------------------------------- < ------------------------------------ #

<h1>LOAD DATA</h1>

Removed file: /home/gr00stl/Nextcloud/Projects/Python/Flask/polish_budget_viz/output_data/polish_budget.csv
Created file: /home/gr00stl/Nextcloud/Projects/Python/Flask/polish_budget_viz/output_data/polish_budget.csv
Reading data from: /home/gr00stl/Nextcloud/Projects/Python/Flask/polish_budget_viz/data/ZaИеcznik nr 2/ZaИеcznik nr 2 cz od 01 do 82.xls
Removed file: /home/gr00stl/Nextcloud/Projects/Python/Flask/polish_budget_viz/output_data/01_82.csv
Created file: /home/gr00stl/Nextcloud/Projects/Python/Flask/polish_budget_viz/output_data/01_82.csv


/tmp/ipykernel_22682/3449016492.py:248: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  single_output_file = single_output_file.append(df, ignore_index=True)
/tmp/ipykernel_22682/3449016492.py:248: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  single_output_file = single_output_file.append(df, ignore_index=True)


Reading data from: /home/gr00stl/Nextcloud/Projects/Python/Flask/polish_budget_viz/data/ZaИеcznik nr 2/ZaИеcznik nr 2 cz od 84 do 90.xls
Removed file: /home/gr00stl/Nextcloud/Projects/Python/Flask/polish_budget_viz/output_data/84_90.csv
Created file: /home/gr00stl/Nextcloud/Projects/Python/Flask/polish_budget_viz/output_data/84_90.csv


In [12]:
# add column per capita -> Plan / 37 766 327
single_output_file['per_capita'] = single_output_file['Plan'] / 37766327 * 1000
#convert float to int
single_output_file['per_capita'] = single_output_file['per_capita'].astype(int)
#round to two decimals
single_output_file['per_capita'] = single_output_file['per_capita'].round(2)

# combine Treść and per_capita columns into super_label column, unless both are 'total' then use 'total' only
single_output_file['percapita_super_label'] = single_output_file.apply(lambda x: x['Treść'] + ' ' + str(x['per_capita']) if x['Treść'] != 'total' and x['per_capita'] != 'total' else x['Treść'] if x['Treść'] == 'total' else x['per_capita'], axis=1)
# add zł to percapita_super_label
single_output_file['percapita_super_label'] = single_output_file['percapita_super_label'] + ' zł'

# save single output file again
single_output_file.to_csv(
    f'{output_data_path}{single_output_file_name}',
    index=False,
    sep=';'
)